In [10]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import numpy as np
import pandas as pd


from pymongo import MongoClient


###########################
# Data Manipulation / Model
###########################

from animalShelter import AnimalShelter

username = "aacuser"
password = "12345"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')

image_filename = 'GraziosoSalvareLogo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
initial_row = {"row":0,  "column":0}

app.layout = html.Div([
    
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Center(html.H2('Jiwon Lee SNHU CS-340')),
    html.A(href="https://www.snhu.edu/",
            children=[
            html.Center(   
            html.Img(id='customer-image',
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     height=200
                     ))]),
    
    # Button location at top
    html.Div(
        dcc.RadioItems(
            id='radio',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
                ],
            value='Reset'
        )),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable='single',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            #id='map-id',
            #className='col s12 m6', 
            dl.Map(
            [dl.TileLayer(id="base-layer-id"), dl.LayerGroup(id='layer')],
            style={'width': '1000px', 'height': '500px'}, 
            center=[30.75,-97.48], 
            zoom=10,   
            )
            )
        ])

])

#############################################
# Interaction Between Components / Controller
#############################################


# Filtering the data
@app.callback(
        Output('datatable-id',"data"), 
        [Input('radio', 'value')]
)
def update_dashboard(value):
    
    if value == 'Reset':
        df = pd.DataFrame.from_records(shelter.read({}))
   
    if value == 'Water':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog", 
                                             "breed": {"$in": ["Labrador Retriever Mix", 
                                                               "Chesapeake Bay Retriever", 
                                                               "Newfoundland"]}, 
                                             "sex_upon_outcome": "Intact Female",
                                             "age_upon_outcome_in_weeks": {"$gte": 26, "$lte":156}
                                            })))
        
    if value == 'Mountain or Wilderness Rescue':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog",
                                             "breed": 
                                                 {"$in": ["German Shepherd", 
                                                          "Alaskan Malamute",
                                                          "Old English Sheepdog", 
                                                          "Siberian Husky", 
                                                          "Rottweiler"]},
                                             "sex_upon_outcome": "Intact Male",
                                             "age_upon_outcome_in_weeks": {"$gte": 26, "$lte":156}
                                            
                                            })))
    
    if value == 'Disaster or Individual Tracking':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog",
                                            "breed": 
                                                 {"$in": ["Doberman Pinscher", 
                                                          "German Shepherd",
                                                          "Golden Retriever", 
                                                          "Bloodhound", 
                                                          "Rottweiler"]},
                                             "sex_upon_outcome": "Intact Male",
                                             "age_upon_outcome_in_weeks": {"$gte": 20, "$lte":300}
                                            })))                                       
        
    return df.to_dict('records')
    
    
# Highlighting cell
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


    
# Pie chart  
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # imports the currently displayed data
    dff = df if viewData is None else pd.DataFrame(viewData) 
    #dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names (breed) and values (recurring counts)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                width=800, 
                height=500   
            )
        )
    ]

    
    
# Geolocation chart
@app.callback(
    Output('layer', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(trigger, selected):
    
   
    dff = df if selected is None else pd.DataFrame(trigger) 
    
    #a = dff.loc[selected, "location_lat"]
    #b = dff.loc[selected, "location_long"]
    #breed = dff.iloc[selected, 4]
    #name = dff.iloc[selected, 9]
   
    
    return [           
         
            dl.Marker(
                position=[
                    float(dff.iloc[selected, 13].values[0]),
                    float(dff.iloc[selected, 14].values[0]),          
                ],
                children=[
                    dl.Tooltip(dff.iloc[selected, 4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[selected, 9])
                        ])
            ])
        ]
    

app     

